## ✅ Step 1: Load data
Load data from Excel-file `file.xlsx` in DataFrame for analysis.

In [1]:
import pandas as pd
df=pd.read_excel('C:/Users/Acer/Desktop/file.xlsx')

## ✅ Step 2: Understand the Dataset
- `df.info()`: Provides a summary of the DataFrame, including the number of columns, their names, data types, and count of non-null values.
- `df.describe()`: Generates statistical summaries for numeric columns, including count, mean, standard deviation, minimum, maximum, and quartiles (25%, 50%, 75%).
- `df.head()`: Displays the first 5 rows of the DataFrame for visual inspection.
- `df.shape`: Returns the dimensions of the DataFrame (number of rows and columns).
- `df.columns`: Lists the names of all columns in the DataFrame.

In [2]:
# Display DataFrame structure and data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 479 entries, 0 to 478
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0    Cust ID        452 non-null    object 
 1    Date           479 non-null    object 
 2   Product         479 non-null    object 
 3    Quantity       455 non-null    float64
 4   Unit Price      461 non-null    float64
 5   Payment Method  451 non-null    object 
 6    Comment        350 non-null    object 
 7    Amount         437 non-null    float64
dtypes: float64(3), object(5)
memory usage: 30.1+ KB


In [3]:
# Generate statistical summary for numeric columns
df.describe()

,Quantity,Unit Price,Amount
count,455.000000,461.000000,437.000000
mean,2.017582,455.314534,1623.729103
std,0.799035,368.116473,5033.686181
min,1.000000,50.000000,50.000000
25%,1.000000,200.000000,300.000000
50%,2.000000,350.000000,700.000000
75%,3.000000,600.000000,1500.000000
max,3.000000,1500.000000,71043.392545


In [4]:
# Show the first 5 rows for inspection
df.head()

,Cust ID,Date,Product,Quantity,Unit Price,Payment Method,Comment,Amount
0,C063,2025-07-28,Phone (budget),1.0,300.0,Credit card,NaN,300.0
1,C021,2025-07-30,Phone (budget),2.0,300.0,online,Note,600.0
2,C005,2025-06-15,Smartwatch (premium),1.0,350.0,online,Note,350.0
3,C066,2025-07-13,Headphones (medium),3.0,100.0,online,Special,300.0
4,C069,2025-07-31,Phone (premium),3.0,800.0,Credit card,Note,2400.0


In [5]:
# Check the dimensions of the DataFrame
df.shape

(479, 8)

In [6]:
# List all column names
df.columns

Index([' Cust ID ', ' Date ', 'Product ', ' Quantity', 'Unit Price ',
       'Payment Method', ' Comment ', ' Amount'],
      dtype='object')

### ❓ Explanation:
- The dataset contains 479 rows and 8 columns.
- Missing values are present in several columns: `Cust ID` (27 missing), `Quantity` (24 missing), `Unit Price` (18 missing), `Payment Method` (28 missing), `Comment` (129 missing), and `Amount` (42 missing).
- The `Date` column is of type `object` and will need to be converted to `datetime` in later steps.
- Numeric columns (`Quantity`, `Unit Price`, `Amount`) show reasonable ranges, but `Amount` has a significantly high maximum value (71043.39 compared to the 75% quartile of 1500), indicating potential outliers.
- Column names contain leading/trailing spaces (e.g., ` Cust ID `, ` Date `), which will be addressed in the data cleaning process.

## ✅ Step 3: Clean Column Names
We clean and standardize column names for consistency:
- Remove leading/trailing spaces using `str.strip()`.
- Convert names to lowercase and replace spaces with underscores using `str.lower()` and `str.replace()`.
- Rename `cust_id` to `customer_id` for clarity.

In [7]:
# Remove leading/trailing spaces from column names
df.columns = df.columns.str.strip()

# Convert column names to lowercase and replace spaces with underscores
df.columns = df.columns.str.lower().str.replace(' ', '_')

# Rename 'cust_id' to 'customer_id' for consistency
df.rename(columns={'cust_id': 'customer_id'}, inplace=True)

# Verify column names
print(df.columns)

Index(['customer_id', 'date', 'product', 'quantity', 'unit_price',
       'payment_method', 'comment', 'amount'],
      dtype='object')


### ❓ Explanation:
- Column names are now standardized: `customer_id`, `date`, `product`, `quantity`, `unit_price`, `payment_method`, `comment`, `amount`.
- All spaces have been removed, and names are in lowercase with underscores for consistency.

## ✅ Step 4: Fix Data Types

To ensure accurate date-based operations in subsequent steps, we standardize the `date` column by converting it to a consistent `datetime` format. The dataset contains mixed date formats (e.g., `2025-07-20 00:00:00` and `2025-07-20`), which require flexible parsing. We aim to remove the time component and handle invalid entries gracefully.

In [8]:
from dateutil import parser
import datetime

def force_parse_date(x):
    if isinstance(x, datetime.date):
        return x
    try:
        return parser.parse(str(x)).date()
    except:
        return None

df['date'] = df['date'].apply(force_parse_date)

# Verify data types and check for missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 479 entries, 0 to 478
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   customer_id     452 non-null    object 
 1   date            479 non-null    object 
 2   product         479 non-null    object 
 3   quantity        455 non-null    float64
 4   unit_price      461 non-null    float64
 5   payment_method  451 non-null    object 
 6   comment         350 non-null    object 
 7   amount          437 non-null    float64
dtypes: float64(3), object(5)
memory usage: 30.1+ KB


### ❓ Explanation:
- We define a custom function `force_parse_date` using the `dateutil.parser` library to parse various date formats into `datetime.date` objects, stripping any time components.
- The function checks if the input is already a `datetime.date` object to avoid redundant processing, attempts to parse string inputs, and returns `None` for unparseable values.
- The `apply` method processes each value in the `date` column, ensuring all valid dates are standardized.
- We verify the results using `df.info()` to check the data type and remaining missing values, and inspect rows with `None` to identify any persistent issues.

## ✅ Step 5: Remove Duplicates
We remove duplicate rows to ensure each record is unique:
- Use `df.drop_duplicates()` to delete fully identical rows.
- Verify the new dataset size with `df.shape`.

In [9]:
print(df.shape)
# Remove duplicate rows
df.drop_duplicates(inplace=True)

# Verify the new shape of the DataFrame
print(df.shape)

(479, 8)
(477, 8)


### ❓ Explanation:
- Removed 2 duplicate rows.
- The dataset is now free of duplicate records, ready for further cleaning.

## ✅ Step 6: Check Missing Values
Missing values in `date`, `amount`, and other columns can affect analysis. We quantify these gaps to guide imputation strategies, confirming the success of date parsing from Step 4.

In [10]:
# Check missing values
print(df.isnull().sum())

# Inspect rows with missing 'date'
print("Rows with missing 'date':")
print(df[df['date'].isna()][['customer_id', 'date', 'product', 'payment_method', 'comment', 'amount']])

customer_id        27
date                0
product             0
quantity           24
unit_price         18
payment_method     28
comment           129
amount             42
dtype: int64
Rows with missing 'date':
Empty DataFrame
Columns: [customer_id, date, product, payment_method, comment, amount]
Index: []


### ❓ Observations:
- Missing values: `customer_id` (27), `date` (0), `quantity` (24), `unit_price` (18) `payment_method` (28), `comment` (129), `amount` (42).
- No missing values in `date`, confirming successful parsing in Step 4.
- High missing rates in `comment` and `amount` highlight areas for imputation in Step 8.

## ✅Step 7: Check Outliers
Outliers in `quantity` or `unit_price` may indicate data errors that contribute to incorrect `amount` values, such as 71043.39. We use the Interquartile Range (IQR) method to identify these anomalies, preserving all transactions for correction in Step 8 to maintain the integrity of sales data.

In [11]:
# Calculate IQR for 'quantity'
Q1_qty = df['quantity'].quantile(0.25)
Q3_qty = df['quantity'].quantile(0.75)
IQR_qty = Q3_qty - Q1_qty
lower_bound_qty = Q1_qty - 1.5 * IQR_qty
upper_bound_qty = Q3_qty + 1.5 * IQR_qty

# Calculate IQR for 'unit_price'
Q1_price = df['unit_price'].quantile(0.25)
Q3_price = df['unit_price'].quantile(0.75)
IQR_price = Q3_price - Q1_price
lower_bound_price = Q1_price - 1.5 * IQR_price
upper_bound_price = Q3_price + 1.5 * IQR_price

# Identify outliers
outliers = df[(df['quantity'] < lower_bound_qty) | (df['quantity'] > upper_bound_qty) |
              (df['unit_price'] < lower_bound_price) | (df['unit_price'] > upper_bound_price)]
print("Outliers in 'quantity' or 'unit_price':")
print(outliers[['quantity', 'unit_price', 'amount']])
print()
# No removal, keep all rows
print("Dataset shape:", df.shape)

Outliers in 'quantity' or 'unit_price':
     quantity  unit_price        amount
23        1.0      1500.0   1500.000000
87        3.0      1500.0   4500.000000
115       2.0      1500.0   3000.000000
125       1.0      1500.0   1500.000000
127       1.0      1500.0   1500.000000
170       NaN      1500.0           NaN
203       2.0      1500.0   3000.000000
219       2.0      1500.0   3000.000000
225       3.0      1500.0   4500.000000
239       1.0      1500.0   1500.000000
251       2.0      1500.0   3000.000000
269       2.0      1500.0   3000.000000
272       1.0      1500.0   1500.000000
273       2.0      1500.0   3000.000000
282       3.0      1500.0   4500.000000
287       3.0      1500.0   4500.000000
303       1.0      1500.0   1500.000000
351       3.0      1500.0  71043.392545
360       2.0      1500.0   3000.000000
392       3.0      1500.0   4500.000000
403       3.0      1500.0   4500.000000
420       3.0      1500.0   4500.000000
442       2.0      1500.0   3000.000000


### ❓ Explanation:
- Identified 26 outliers in `quantity` or `unit_price`, primarily with `unit_price = 1500.0` and `quantity` ranging from 1.0 to 3.0.  
- Most outliers were valid transactions, with `amount` aligning with `quantity` * `unit_price` (e.g., 3.0 * 1500.0 = 4500.0), except for errors like `amount = 71043.39`.  
- Two outliers had missing `quantity` values, consistent with the 24 missing entries in `quantity`.  
- No rows were removed, preserving all 477 rows and 8 columns to avoid losing valid sales data.  
- Errors in `amount` will be corrected in Step 8 by recalculating `quantity` * `unit_price` for all rows.  

## ✅ Step 8: Handle Missing Values
Missing values in `quantity`, `unit_price`, `amount`, and other columns require imputation. We recalculate `amount` for all rows to correct errors and ensure data consistency.

In [12]:
# Fill missing values
df['quantity'] = df['quantity'].fillna(df['quantity'].mean())
df['unit_price'] = df['unit_price'].fillna(df['unit_price'].mean())
df['customer_id'] = df['customer_id'].fillna(df['customer_id'].mode()[0])
df['payment_method'] = df['payment_method'].fillna(df['payment_method'].mode()[0])
df['comment'] = df['comment'].fillna('No Comment')
df['date'] = df['date'].fillna(df['date'].mode()[0])

# Recalculate 'amount' for all rows
df['amount'] = df['quantity'] * df['unit_price']

# Convert 'date' to datetime64
df['date'] = pd.to_datetime(df['date'])

# Verify no missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 477 entries, 0 to 478
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   customer_id     477 non-null    object        
 1   date            477 non-null    datetime64[ns]
 2   product         477 non-null    object        
 3   quantity        477 non-null    float64       
 4   unit_price      477 non-null    float64       
 5   payment_method  477 non-null    object        
 6   comment         477 non-null    object        
 7   amount          477 non-null    float64       
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 33.5+ KB


### ❓ Observations:  
**Observations**:
- Filled missing values: `customer_id` (27), `quantity` (24), `unit_price` (18), `payment_method` (28), `comment` (129), `date` (0, prophylactically).
- Recalculated `amount` for all 477 rows, correcting errors like 71043.39.
- Converted `date` from `object` to `datetime64` for Step 12.
- No missing values remain, dataset has 477 rows and 8 columns.

## ✅ Step 9: Fix Label Inconsistencies
Inconsistent labels in the `payment_method` column, such as variations in case ('CC' vs 'cc') or abbreviations ('online' vs 'Online'), can lead to errors in analysis. We standardize these values to ensure consistency and facilitate accurate grouping in subsequent steps.

In [13]:
df['payment_method'] = df['payment_method'].str.lower().replace({'cc': 'credit_card', 'online': 'online_payment'})
print(df['payment_method'].value_counts())

payment_method
credit card       203
cash              178
online_payment     96
Name: count, dtype: int64


### ❓ Explanation:  
**Observations**:
- Standardized `payment_method` values, resulting in three categories: `credit card` (203), `cash` (178), `online_payment` (96).
- Eliminated case variations and abbreviations, ensuring consistency across all 477 rows.
- Verified with `value_counts()` that no missing or inconsistent labels remain.

## ✅ Step 10: Drop Unnecessary Columns
The `comment` column, with approximately 27% missing values filled with 'No Comment' in Step 8, provides limited analytical value. We remove it to streamline the dataset and focus on columns critical for subsequent analysis.

In [14]:
df = df.drop(columns=['comment'])
print(df.columns)

Index(['customer_id', 'date', 'product', 'quantity', 'unit_price',
       'payment_method', 'amount'],
      dtype='object')


### ❓ Explanation:
- Removed the `comment` column, which had 129 missing values filled with 'No Comment' in Step 8.  
- Reduced the dataset to 7 columns: `customer_id`, `date`, `product`, `quantity`, `unit_price`, `payment_method`, and `amount`.  
- Verified with `df.columns` that the dataset, retaining 477 rows, is now streamlined for efficient analysis in subsequent steps.  

## ✅ Step 11: Create New Columns
To validate the `amount` column, we create a new column to check if `amount` equals `quantity` * `unit_price`, ensuring data consistency.

In [15]:
df['amount_check'] = df['quantity'] * df['unit_price']
print(df[['quantity', 'unit_price', 'amount', 'amount_check']].head(10))

   quantity  unit_price  amount  amount_check
0       1.0       300.0   300.0         300.0
1       2.0       300.0   600.0         600.0
2       1.0       350.0   350.0         350.0
3       3.0       100.0   300.0         300.0
4       3.0       800.0  2400.0        2400.0
5       2.0       600.0  1200.0        1200.0
6       1.0       200.0   200.0         200.0
7       3.0       350.0  1050.0        1050.0
8       1.0       600.0   600.0         600.0
9       2.0       300.0   600.0         600.0


### ❓ Explanation:
- We calculate `amount_check` as `quantity` * `unit_price` to verify the `amount` column's accuracy.
- A sample of rows is displayed to compare `amount` and `amount_check`, identifying any discrepancies.

## ✅ Step 12: Split Date into Year and Month
To facilitate time-based analysis, we extract the year and month from the `date` column into separate columns.

In [16]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
print(df[['date', 'year', 'month']].head())

        date  year  month
0 2025-07-28  2025      7
1 2025-07-30  2025      7
2 2025-06-15  2025      6
3 2025-07-13  2025      7
4 2025-07-31  2025      7


### ❓ Explanation: 
- We use `dt.year` and `dt.month` to extract the year and month from the `date` column.
- A sample of rows is displayed to confirm the new columns are correctly populated.

## ✅ Step 13: Save Cleaned Data
The cleaned and processed dataset is saved for further analysis or sharing, ensuring all transformations are preserved.

In [17]:
df.to_csv('C:/Users/Acer/Desktop/cleaned_data.csv', index=False)
print("Data saved to 'cleaned_data.csv'")
print()
# Display DataFrame structure and data types
df.info()

Data saved to 'cleaned_data.csv'

<class 'pandas.core.frame.DataFrame'>
Index: 477 entries, 0 to 478
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   customer_id     477 non-null    object        
 1   date            477 non-null    datetime64[ns]
 2   product         477 non-null    object        
 3   quantity        477 non-null    float64       
 4   unit_price      477 non-null    float64       
 5   payment_method  477 non-null    object        
 6   amount          477 non-null    float64       
 7   amount_check    477 non-null    float64       
 8   year            477 non-null    int32         
 9   month           477 non-null    int32         
dtypes: datetime64[ns](1), float64(4), int32(2), object(3)
memory usage: 37.3+ KB


### ❓ Explanation:
- We use `df.to_csv()` to save the DataFrame to a CSV file named `cleaned_data.csv`.
- The `index=False` parameter prevents saving the DataFrame index as a column.
- A confirmation message verifies the operation.